In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pdb

import meerkat as mk
from meerkat.contrib.eeg import build_stanford_eeg_dp
from domino.utils import split_dp, balance_dp

from sklearn.metrics import precision_score, recall_score, roc_auc_score


## Load EEG datapanel

In [ ]:
dp_art = build_stanford_eeg_dp.out(run_id=618) # with text constraint: 409
dp=dp_art.load()
dp.lz[0]

In [ ]:
len(dp)

In [ ]:
balanced_dp_art = balance_dp.out(623) # with text constraint: 622
balanced_dp = balanced_dp_art.load()

In [ ]:
len(balanced_dp)

## split data

In [ ]:
dp_splits_art = split_dp.out(625) #split_dp(balanced_dp_art, split_on="file_id")
dp_splits = dp_splits_art.load()
print(len(dp_splits))
dp_splits.head()

In [ ]:
thresh = 1
age_labels = balanced_dp["age"] > thresh
sz_labels = balanced_dp["target"]
np.corrcoef([age_labels,sz_labels])[1,0]

## Slice dp based on metadata

In [ ]:
from domino.slices.eeg import EegSliceBuilder

dp_age = EegSliceBuilder().build_correlation_slices(balanced_dp, correlate="age", corr=0.5, n=8000, correlate_threshold=thresh)

In [ ]:
len(dp_age)

In [ ]:
age_labels = dp_age["age"] > thresh
sz_labels = dp_age["target"]
np.corrcoef([age_labels,sz_labels])[0,1]

## Get multiple slices

In [ ]:
from domino.slices.eeg import collect_correlation_slices

# correlate_list = ["age"]
# corr_list = [0, 0.3, 0.5, 0.9]
# correlate_thresholds = [10]
# dp_slices_art = collect_correlation_slices(correlate_list, corr_list, correlate_thresholds)
dp_slices_art = collect_correlation_slices.out(516)
dp_slices = dp_slices_art.load()
dp_slices.head()

In [ ]:
from domino.train import score_slices, train_slices, train_model

In [ ]:
import matplotlib.pyplot as plt

metric = "auroc"

scores_dp = score_slices.out(652).load() # on valid: 627, on test: 595
plt.plot(scores_dp["corr"].data, scores_dp[f"in_slice_0_{metric}"].data, color="green")
#plt.scatter(scores_dp["corr"].data, scores_dp[f"in_slice_0_{metric}_mean"].data, color="green")
#plt.fill_between(scores_dp["corr"].data, scores_dp[f"in_slice_0_{metric}_lower"].data, scores_dp[f"in_slice_0_{metric}_upper"].data, alpha=0.3, color="green")

plt.plot(scores_dp["corr"].data, scores_dp[f"out_slice_{metric}"].data, color="red")
#plt.scatter(scores_dp["corr"].data, scores_dp[f"out_slice_{metric}_mean"].data, color="red")
#plt.fill_between(scores_dp["corr"].data, scores_dp[f"out_slice_{metric}_lower"].data, scores_dp[f"out_slice_{metric}_upper"].data, alpha=0.3, color="red")

plt.plot(scores_dp["corr"].data, scores_dp[f"overall_{metric}"].data, color="blue")
#lt.scatter(scores_dp["corr"].data, scores_dp[f"overall_{metric}_mean"].data, color="blue")
#plt.fill_between(scores_dp["corr"].data, scores_dp[f"overall_{metric}_lower"].data, scores_dp[f"overall_{metric}_upper"].data, alpha=0.3, color="blue")



plt.legend(["C = Y", "C != Y","overall"])
plt.ylabel(f"mean {metric}")
plt.xlabel("correlation strength")
plt.title("EEG seizure prediction, age slicing")
plt.show()

In [ ]:
scores_dp[f"in_slice_0_{metric}"].data

In [ ]:
scores_dp[f"out_slice_{metric}"].data

In [ ]:
scores_dp[f"overall_{metric}"].data

In [ ]:
dp_ = train_model.inp(578)["dp"].load()
mask = np.logical_and((dp_["slices"].data[:,0]==1),dp_.lz["split"]=="valid")

dp_["target","binarized_age"].lz[mask][-10:]
dp_["target"].lz[mask].sum()

In [ ]:
mask.sum()